In [63]:
import pandas as pd
import numpy as np
import math
import os

#from IPython.display import display

# Made by Henric Pietro Vicente Gil please credit me in your work :) 
# henricgil@discente.ufg.br

In [64]:
"""
# This cell defines the location of the files, 

# path_to_files may be a folder where multiple files are;

# file_list is a list of all the files in the folder provided in path_to_files;

# s_file represents the file(s) to be read, can be either a list or a string

# uncomment the part that you will use, comment the part you wont
"""

# Points to a file ./myfolder/example.xlsx or to a folder c:\\my\\folder\\where\\my\\\files\\are
path_to_files = r'D:\Projetos academicos\Labmol\Code\Data Treatment\data\cli'#r'D:\Projetos academicos\Labmol\Code\Data\My_data\Pre Clinical Data\Legacy_Data\Filtered_Clinical_Data.xlsx'
# if its a single file pass its name
result_name = "LOAEL_clinical_hepato"
try:
    os.mkdir("./data")
except FileExistsError:
    pass
#### To get from a directory

try:
    file_list = [f for f in os.listdir(path_to_files) if os.path.isfile(f"{path_to_files}\\{f}")]
    s_file = [f"{path_to_files}\\{file}" for file in file_list]

#### To get from a path
except NotADirectoryError:
    s_file = path_to_files

In [65]:
"""
# Defines the necessary variables used all through out the code

# duplicate_identifier_column is the collumn that identifies the duplicates in your code

# values_col defines the column where your dose/lc50/ic50... values are

# max_z_score is the z value used to separate the outliers from non outliers

# convert_to_p despite the name, this bool conditions the result into a -log(result) or pResult

# convert_measure bool that conditions if you need to convert measurements, also eliminates impossible values
"""
duplicate_identifier_column = 'final_smiles_stand'
differenciator = "Clinical_"
values_col = "Dose"
max_z_score = 0.3
standardize_mol = False
convert_to_p = False # p = -log(measure)
convert_measure = False

### Standardize thy molecules

In [66]:
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.rdmolops import FindPotentialStereo
from chembl_structure_pipeline import standardizer 
from rdkit.Chem.SaltRemover import SaltRemover

def standardize(df:pd.DataFrame,name):
    name_smiles = "Chemical Structure"
    METALS = ["B","Li","Na","K","Rb","Cs","Be","Mg","Ca","Sr","Ba","Ra","Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Al","Ga","Y","Zr","Nb","Mo","Tc","Ru","Rh","Pd","Ag","Cd","In","Sn","Hf","Ta","W","Re","Os","Ir","Pt","Au","Hg","Tl","Pb","Bi"]
    #NON_METALS = ["Br","Cl","F","Br-","Cl-","F-","Br--","Cl--","F--","B+","Cl+","F+","Br++","Cl++","F++","O","O-","O--","O+","O++"]
    ORGANIC_SALTS = ["OC( = O)\C = C/C(O) = O","C1CCCCC1[NH]C1CCCCC1","OC( = O)C(O)C(O)C( = O)O","OS(O)( = O) = O","OC( = O)C( = O)O","OC( = O)C = CC( = O)O","FC(F)(F)C( = O)O","[CH3]C( = O)O","[N]( = O)(O)O","[P]( = O)(O)(O)O","[P](F)(F)(F)(F)(F)F","[S]( = O)( = O)(O)O","[CH3][S]( = O)( = O)(O)","c1cc([CH3])ccc1[S]( = O)( = O)(O)"]
    #df.dropna(subset = name_smiles,how = "any",axis = 0,inplace = True)
    fonte = "Source"
    removals = []
    
    def remove_non_str(df:pd.DataFrame):
            invalids = 0
            for i,smi in enumerate(df[name_smiles]):
                    if type(smi) is not str:
                        invalids+= 1
                        #df.drop(index = i,axis = 0 ,inplace = True)
                    # try:
                    #     smiles = df[name_smiles][i]
                    #     m = Chem.MolFromSmiles(smiles)
                    #     if m is None:
                    #         invalids+= 1
                    #         df.drop(index = i,axis = 0 ,inplace = True)
                    # except TypeError:
                    #     invalids+= 1
                    #     df.drop(index = i,axis = 0 ,inplace = True)
                        nf = df.loc[i,[name_smiles,fonte]].values.flatten().tolist()
                        removals.append([f"Index: {i}",nf[0],nf[1],"Empty or not an smiles entry"])
                        df.drop(index = i,axis = 0 ,inplace = True)
            df.reset_index(drop = True, inplace = True)
            return df,invalids

    def remove_invalid(df:pd.DataFrame):
        invalids = 0
        for i,smi in enumerate(df[name_smiles]):
            #if type(smi) is not str:    
            try:
                m = Chem.MolFromSmiles(smi)
                if m is None:
                    invalids+= 1
                    nf = df.loc[i,[name_smiles,fonte]].values.flatten().tolist()
                    removals.append([nf[0],nf[1],"invalid"])
                    df.drop(index = i,axis = 0 ,inplace = True)
            except TypeError:
                    invalids+= 1
                    nf = df.loc[i,[name_smiles,fonte]].values.flatten().tolist()
                    removals.append([nf[0],nf[1],"invalid"])
                    df.drop(index = i,axis = 0 ,inplace = True)
            df.reset_index(drop = True, inplace = True)
            return df,invalids

    def remove_metals(df:pd.DataFrame):
        #badAtoms = Chem.MolFromSmarts('[!$([#1,#3,#11,#19,#4,#12,#20,#5,#6,#14,#7,#15,#8,#16,#9,#17,#35,#53])]')
        hm_metal = 0
        hm_mixture = 0
        for i,smiles in enumerate(df[name_smiles]):
            #smiles = df[i]
            if type(smiles) is str:
                """6"""
                if smiles.find(".")!= -1:
                    is_mixture = str(smiles).split(".")
                    non_metal_index = []
                    for n,fragment in enumerate(is_mixture):
                        for metal in METALS:
                            striped = fragment.strip("+-[] ")
                            #print(len(striped))
                            if len(striped)>2 and striped.find(metal)==-1:# and striped not in NON_METALS and striped not in METALS:
                                    if n not in non_metal_index:
                                        non_metal_index.append(n)
                                    else:
                                        continue
                            else:
                                continue
                    #print(len(non_metal_index))
                    try:
                        if len(non_metal_index)>1: #and (is_mixture[non_metal_index[0]]).strip("-+[] ") not in NON_METALS and (is_mixture[non_metal_index[0]]).strip("-+[]") not in METALS:
                            subs = 0
                            for sub in is_mixture:
                                if sub not in ORGANIC_SALTS:
                                    subs+= 1
                            if subs>1:
                                nf = df.loc[i,[name_smiles,fonte]].values.flatten().tolist()
                                removals.append([f"Index: {i}",nf[0],nf[1],"Mixture"])
                                df.drop(index = i,axis = 0,inplace = True)
                                hm_mixture+= 1
                            else:
                                df.loc[i,name_smiles] = is_mixture[subs]
                                
                        elif len(non_metal_index)==1:

                            df.loc[i,name_smiles] = is_mixture[non_metal_index[0]]
                        
                        elif len(non_metal_index)==0:
                            hm_metal+= 1
                            nf = df.loc[i,[name_smiles,fonte]].values.flatten().tolist()
                            removals.append([f"Index: {i}",nf[0],nf[1],"Salt or metal"])
                            df.drop(index = i,axis = 0,inplace = True)
                    except KeyError:
                        continue
                else:
                    try:
                        for metal in METALS:
                            m = smiles.find(metal)
                            if m!= -1 and smiles[m+1] !=  "r":
                                hm_metal+= 1
                                nf = df.loc[i,[name_smiles,fonte]].values.flatten().tolist()
                                removals.append([f"Index: {i}",nf[0],nf[1],"organometal"])
                                df.drop(index = i,axis = 0,inplace = True)
                                break
                    except KeyError:
                        continue    
        df.reset_index(drop = True, inplace = True)
                            
        #df.reset_index(drop = True, inplace = True)                    
        return df,hm_metal,hm_mixture

    def no_stereoisomer_info(df:pd.DataFrame):
        mols = []
        for smi in df[name_smiles]:
            smi = str(smi).replace("@","")
            mols.append(smi)
        no_stereoisomer = pd.DataFrame(mols, columns = ["no_stereoisomer"])
        df_stereoisomer = df.join(no_stereoisomer)
        return df_stereoisomer

    # Removes stereochemistry info and adds it to "no_stereo"
    def chemblStandardizer(df:pd.DataFrame):
        mols = [Chem.MolFromSmiles(smile) for smile in df[name_smiles] if smile is str]
        invalids = 0
        for i,mol in enumerate(mols):
            try:
                smiles = Chem.MolToSmiles(standardizer.standardize_mol(mol))
                df.loc[i,name_smiles] = smiles    
            except:
                try:
                    nf = df.loc[i,[name_smiles,fonte]].values.flatten().tolist()
                    removals.append([f"Index: {i}",nf[0],nf[1],"Even Landrum couldnt do it"])
                    df.drop(index = i,axis = 0 ,inplace = True)
                    invalids+= 1
                except KeyError:
                    continue
        df.reset_index(drop = True, inplace = True)
        return df,invalids

    def standardize_with_and_out_stereo(df) -> pd.DataFrame:
        df,invalids = remove_non_str(df)
        df,real_invalids = chemblStandardizer(df)
        df,hm_metal,hm_mixture = remove_metals(df)
        df = no_stereoisomer_info(df)
        invalids = invalids+real_invalids
        removals_df = pd.DataFrame(removals,columns = ["Index on that table",fonte,name_smiles,"What Happened"],index = range(len(removals)))
        removals_df.to_csv(f"./data/{name}_Removals.csv")
        return df,hm_metal,hm_mixture,invalids#,norm,neutr,taut

    
    #generates mol from smiles
    
    
    return standardize_with_and_out_stereo(df)

##### DO NOT RUN UNLESS YOUR DATA CONTAINS NULL CHARS ########


######### DO NOT RUN UNLESS YOUR DATA CONTAINS NULL CHARS ################

for file in s_file:        
        with open(file,"r") as f:
                a = f.read()

        a = a.replace("\x00","")
        
        with open(file,"w") as f:
                f.write(a)
        # df = pd.read_csv(file,delimiter = ",")
        # print(df.head(1))

##### DONT RUN UNLESS YOUR FILES HAVE SOME COMPATIBILITY ISSUES #######


###### DONT RUN UNLESS YOUR FILES HAVE SOME COMPATIBILITY ISSUES #######
if type(s_file) is not str:
    for file in s_file:
        name = str(file).replace(path_to_files,"")
        name = name.replace(".csv",".xlsx")
        
        df = pd.read_csv(file,delimiter = ",")
        df.to_excel(f"temp_dataset/{name}")

path_to_files = "./temp_dataset"
file_list = os.listdir(path_to_files)
s_file = [f"{path_to_files}\\{file}" for file in file_list]

### Defintively run

In [67]:
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
import re
# e.g. cysteine
#mol = Chem.MolFromSmiles("C([C@@H](C( = O)O)N)S")
#CalcMolFormula(mol)
def remove_non_str(df:pd.DataFrame):
            name_smiles = duplicate_identifier_column
            invalids = 0
            for i,smi in enumerate(df[name_smiles]):
                    if type(smi) is not str:
                        invalids+= 1
                        #df.drop(index = i,axis = 0 ,inplace = True)
                    # try:
                    #     smiles = df[name_smiles][i]
                    #     m = Chem.MolFromSmiles(smiles)
                    #     if m is None:
                    #         invalids+= 1
                    #         df.drop(index = i,axis = 0 ,inplace = True)
                    # except TypeError:
                    #     invalids+= 1
                    #     df.drop(index = i,axis = 0 ,inplace = True)
                        #nf = df.loc[i,[name_smiles,fonte]].values.flatten().tolist()
                        #removals.append([f"Index: {i}",nf[0],nf[1],"Empty or not an smiles entry"])
                        df.drop(index = i,axis = 0 ,inplace = True)
            df.reset_index(drop = True, inplace = True)
            return df#,invalids    
def main(files):
    
    def get_mass(formula):

        parts = re.findall("[A-Z][a-z]?|[0-9]+", formula)
        mass = 0

        for index in range(len(parts)):
            if parts[index].isnumeric():
                continue

            atom = Chem.Atom(parts[index])
            multiplier = int(parts[index + 1]) if len(parts) > index + 1 and parts[index + 1].isnumeric() else 1
            mass +=  atom.GetMass() * multiplier

        return mass
    
    def isInt(i):
        """Check if it is an integer"""
        try:
            int(i)
            return True
        except ValueError:
            return False

    def isFloat(f):
        try:
            float(f)
            return True
        except ValueError:
            return False

    def convertTo_mg(df, column = values_col):
        """
        Pass a pandas 'series like' (aka: df.column) of measurement strings, 
        convert the values into miligram numbers, 
        returns a list of lists with a number and the unit
        
        """

        micro = "\u00B5"
        measure = []

        for dose in df.loc[:, column]:
            # num = [n for n in number if isInt(n) or n=="."]
            num = ""
            unit = ""
            nums = []
            avg = 0
            # Ensures it is not nan or None
            if type(dose) is str:
                for i, n in enumerate(dose):
                    # Avoids unprecise entries
                    if n in ["<", ">"]:
                        continue

                    if isInt(n) or n == "." and num.find(".") == -1:
                        num +=  n

                    if isInt(n) == False:
                        unit +=  n
                        num = ""
                    if n in [",", "-", "\\", ".", "/"] and num.find(".") !=  0 and num !=  "":
                        num.strip(' ,-\\/[]"')
                        nums.append(float(num))
                        num = ""
                    if len(nums) > 1 and "" not in nums:
                        avg = np.mean(nums)

                    elif len(nums) == 0 and num !=  "":
                        avg = float(num)
                # unit = [str(s) for s in number if isInt(s)==False]
                measure.append([avg, str(unit).strip(" -.")])

            elif type(dose) is float or type(dose) is int:
                measure.append([dose, "mg"])
        for i, m in enumerate(measure):
            # print(m)
            u = str(m[1])
            n = m[0]

            # if you want to use the strings that contain unit values uncomment the commented lines
            if u.find("kg") !=  -1 and u.find("mg") == -1:
                measure[i][0] = n*1_000_000
                # measure[i][1] = str(measure[i][1]).replace("kg","mg")

            # might be with special char
            elif u.find(f"{micro}g") !=  -1 or unit.find("ug") !=  -1:
                measure[i][0] = n/1000
                # measure[i][1] = str(measure[i][1]).replace("ug","mg")

            elif u.find("cg") !=  -1:
                measure[i][0] = n*10
                # measure[i][1] = str(measure[i][1]).replace("cg","mg")
            if u.find("mg") == 0:
                measure[i][0] = n

            if u.find("g") == 0:
                measure[i][0] = n*1000
                # measure[i][1] = str(measure[i][1]).replace("g","mg")

        solved_col = pd.Series(measure)

        return solved_col
    
    # Method that calculates the standard deviation with a number list and the mean, but use np.std() instead
    def stdCalculation(numList, mean):
        n = len(numList)+1
        soma = 0
        for x in numList:
            soma += (x-mean)**2
        std = math.sqrt((soma)/n)
        return std

    # Method to calculate the z score for each number in a array of numbers, returns a list of z-scores of each number relevant to the array
    def z_scorer(nums: list, mean: float or int, std: float or int):

        z_scores = []
        for x in nums:
            z = (x-mean)/std
            z_scores.append(z)
        return z_scores

    # Most of the action happens here
    def idOutliers(df: pd.DataFrame, name_col: str = "Drug", value_col: str = "Dose", max_z: float = 1.8, convert_to_p: bool = True):
        dict_rows = {}
        means = {}
        # Comment if you have empty values
        df = df.dropna(axis = 0, how = "all")
        
        if convert_measure:
            values = [float(num) for num,mg in df.loc[:, value_col]]
           # print(values)
        else:
            values = [float(num) for num in df.loc[:, value_col]]
        
        names = [str(drugs) for drugs in df.loc[:, name_col]]# if type(drugs) is str]
        mols = [Chem.MolFromSmiles(smi) for smi in names]
        formulas = [CalcMolFormula(mol) for mol in mols if mol is not None]
        for i, name in enumerate(names):
            if name=="" or name=="-":
                name = "Empty"
                
            if name not in dict_rows:
                dict_rows[name] = []
                
                dict_rows[name].append(values[i])
            else:
                dict_rows[name].append(values[i])

        for i,n in enumerate(dict_rows):
            media = []
            molw = get_mass(formulas[i])

            if len(dict_rows[n]) > 1 and type(dict_rows[n]) is float:
                means[n] = np.mean(dict_rows[n])

                if dict_rows[n][0] != 0 and means[n]/dict_rows[n][0] !=  1:
                    std = np.std(dict_rows[n])
                else:
                    std = 1

                z = z_scorer(dict_rows[n], means[n], std)

                for j, z_value in enumerate(z):
                    if abs(z_value) <= max_z:
                        media.append(dict_rows[n][j])

                med = np.mean(media)
                
                # atrubutes where in the column specified is equal to the name of the current row, and replaces the value (dose) of that row
                if convert_to_p:
                    df.loc[df[name_col] == n, value_col] = -math.log10(med/molw)
                else:
                    df.loc[df[name_col] == n, value_col] = med/molw
            else:
                val = dict_rows[n][0]
                #print(dict_rows[n])
                if convert_to_p:
                    if dict_rows[n][0] <=  0:
                        df.loc[df[name_col] == n, value_col] = val
                    else: 
                        df.loc[df[name_col] == n, value_col] = -math.log10(val/molw)
                else:
                    df.loc[df[name_col] == n, value_col] = val/molw
                    
        dups = len(values) - len(dict_rows.keys())
        tl = len(dict_rows.keys())
        return df,dups,tl
    
    # computes in the order necessary to generate the dataframes
    def organize(df,name):
        # read
        print(name)    
        # convert?
        if convert_measure:
            new_doses = convertTo_mg(df, values_col)
            df[values_col] = new_doses
        # generate
        df = remove_non_str(df)
        if standardize_mol:
            df,hm_metal,hm_mixture,invalids = standardize(df,name)
            text = f"Table: {name}: \n Removed metals: {hm_metal} \t Removed mixtures: {hm_mixture} \t Invalids removed: {invalids} \t Removed duplicates: {dups} \t Total Lenght: {total_lenght} \n \n"
        else:
            text = f"Table: {name}" 
        
        z,dups,total_lenght = idOutliers(df, name_col = duplicate_identifier_column, value_col = values_col, max_z = max_z_score, convert_to_p = convert_to_p)
        
        text += f"\t Removed Duplicates: {dups} \t Total Lenght: {total_lenght} \n \n"
        
        z = z.drop_duplicates(keep = "first",subset = duplicate_identifier_column)
        
        if z.shape[0] > 50 :
            z.to_csv(f"./duplicate_analysis/{differenciator}{name}.csv",index = False)
        else:
            print(f"|With {total_lenght}| this |endpoint {name}|  had less than 50 entries  ")
        
        with open(f"{result_name}.txt","a") as f:
            f.write(text)
        print(f"Generated: {name}")

    # Made to work with a single file and a list of files
    def read_through_files(files):

        if type(files) is str:
            name = result_name
            name = name.replace(".xlsx","")
            name = name.replace(".csv","")
            if files.find(".csv")!= -1:
                df = pd.read_csv(files, delimiter = ",") #encoding = 'latin-1')
                organize(df,name)
            else:
                sheets = pd.ExcelFile(files).sheet_names
                if len(sheets)>1:
                    for sheet in sheets:
                        if sheet !=  ".":
                            name = sheet
                            df = pd.read_excel(files,sheet_name = sheet)
                            organize(df,name)
                else:
                    for sheet in sheets:
                        if sheet !=  ".":
                            #name = sheet
                            df = pd.read_excel(files,sheet_name = sheet)
                            organize(df,name)
        else:
            for file in files:
                name = str(file).replace(path_to_files,"")
                name = name.strip("/\\")
                name = name.replace(".xlsx","")
                name = name.replace(".csv","")
                if file.find(".csv") !=  -1:
                    df = pd.read_csv(file,delimiter = ",")#,encoding = 'latin-1')
                    organize(df,name)
                else:   
                    sheets = pd.ExcelFile(file).sheet_names
                    if len(sheets)>1:
                        for sheet in sheets:
                            if sheet !=  ".":
                                name = sheet
                                df = pd.read_excel(file,sheet_name = sheet)
                                organize(df,name)
                    else:
                        for sheet in sheets:
                            if sheet !=  ".":
                                #name = sheet
                                df = pd.read_excel(file,sheet_name = sheet)
                                organize(df,name)

    read_through_files(files = files)

### Run to execute
###### Equivalent to if __name__ == __main__:

In [68]:
# Execute everything
main(s_file)
    

final_Abnormal Hepatic Fun 


UnboundLocalError: local variable 'molw' referenced before assignment